# Config

In [83]:
import os
import logging
from flask import Flask, jsonify # type: ignore
from flask_sqlalchemy import SQLAlchemy # type: ignore
from docx import Document # type: ignore
import pdfplumber # type: ignore
import pandas as pd # type: ignore
import json
import sys
import requests
import re
from docx import Document
from datetime import datetime, timedelta
from sqlalchemy import Column, String, Float, Date, Integer
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [84]:
sales_order_file = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\Daily Update\Open Sales Order 8_12_2025.CSV"
try:
    df_sales_order = pd.read_csv(sales_order_file, encoding="ISO-8859-1")
except UnicodeDecodeError:
    df_sales_order = pd.read_csv(sales_order_file, encoding="latin1")

In [97]:
# Configure Supabase
logging.basicConfig(level=logging.INFO)

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False


db = SQLAlchemy()

# Hook your existing db object up to this app
db.init_app(app)

class ReceivingLog(db.Model):
    __tablename__   = 'receiving_log'
    id = Column(Integer, primary_key=True, autoincrement=True)
    serial_number   = db.Column(db.String(255), primary_key=True)
    entry_date      = db.Column(db.Date)
    invoice_number  = db.Column(db.String(255))
    box_number      = db.Column(db.String(255))
    pod_number      = db.Column(db.String(255))
    part_number     = db.Column(db.String(255))
    quantity        = db.Column(db.Float)
    reference     = db.Column('Reference', db.Text)  

def extract_useful_number(sn: str) -> str:
    """
    Extracts the serial number portion from a string in the format 'SN########'.
    """
    match = re.search(r"SN?([A-Za-z0-9\-]+)", sn)
    return match.group(1) if match else sn.strip()

def get_part_name_from_db(serial_number: str):
    try:
        with app.app_context():
            entry = ReceivingLog.query.filter_by(serial_number=serial_number).first()
            return entry.part_number if entry else None
    except Exception as e:
        logging.error(f"DB error for {serial_number}: {e}")
        return None


def extract_product_details_from_word(file_path):
    try:
        document = Document(file_path)
        if not document.tables:
            return []

        table = document.tables[0]
        product_details = []
        for row in table.rows[1:-1]:
            cells = row.cells
            if len(cells) < 4:
                continue

            sn_text = cells[2].text.strip()
            if sn_text.upper() in {"NA", "N/A", "NONE"} or not sn_text.strip():
                continue  

            product_details.append({
                "product_number": cells[0].text.strip(),
                "qty": cells[1].text.strip(),
                "sn": cells[2].text.strip(),
                "notes": cells[3].text.strip()
            })
        return product_details
    except Exception as e:
        print(f"Error reading Word file {file_path}: {e}")
        return []


def validate_sn_part_matches_via_api(file_path):
    """
    Validates if the serial numbers from the Word file match the database.
    Also checks if the quantity matches the number of serial numbers.
    """
    results = []
    product_details = extract_product_details_from_word(file_path)

    for item in product_details:
        word_part = item["product_number"]
        sn_block = item["sn"]
        qty_text = item["qty"]
        serials = [s.strip() for s in sn_block.split('\n') if s.strip() and s.strip().upper() not in {"NA", "N/A"}]
        sn_count = len(serials)

        try:
            expected_qty = int(qty_text)
        except ValueError:
            expected_qty = None

        qty_match = (sn_count == expected_qty) if expected_qty is not None else False

        if not serials:
            results.append({
                "serial_number": "N/A",
                "word_part": word_part,
                "db_part": None,
                "status": "❓ NOT FOUND",
                "qty_check": f"❌ Qty Mismatch"
            })
            continue

        for sn in serials:
            db_part = get_part_name_from_db(sn)
            if db_part:
                match = "✅ MATCH" if db_part == word_part else "❌ MISMATCH"
            else:
                match = "❓ NOT FOUND"

            results.append({
                "serial_number": sn,
                "word_part": word_part,
                "db_part": db_part,
                "status": match,
                "qty_check": f"✅ Qty OK" if qty_match else f"❌ Qty Mismatch (Expected {expected_qty}, Found {sn_count})"
            })

    return results


def load_receiving_log(path_to_xlsm: str, engine, dry_run=False):
    import pandas as pd

    # Load only the 'Receiving' sheet
    df = pd.read_excel(path_to_xlsm, sheet_name="Receiving")

    # Remove rows where all critical fields are blank (before renaming)
    df = df.dropna(subset=['Date', 'Inv# ', 'Box #', 'POD#', 'Part#', 'SN#'], how='all')

    # Rename columns to standardized names
    df.rename(columns={
        'Date': 'entry_date',
        'Inv# ': 'invoice_number',
        'Box #': 'box_number',
        'POD#': 'pod_number',
        'Part#': 'part_number',
        'SN#': 'serial_number',
        'QTY': 'quantity'
    }, inplace=True)

    # Normalize types and strip spaces
    df['quantity'] = df['quantity'].fillna(1).astype(float)
    df['entry_date'] = pd.to_datetime(df['entry_date'], errors='coerce')
    df['serial_number'] = df['serial_number'].astype(str).str.strip().replace("nan", "NA")

    string_cols = ['invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number']
    for col in string_cols:
        df[col] = df[col].astype(str).str.strip()

    # Print preview after cleanup
    print(f"🧾 Cleaned DataFrame Preview:\n{df.tail(25)}")

    # Load existing data for deduplication
    existing = pd.read_sql("""
        SELECT entry_date, invoice_number, box_number, pod_number, part_number, serial_number, quantity
        FROM receiving_log
    """, engine)

    existing['entry_date'] = pd.to_datetime(existing['entry_date'], errors='coerce')
    for col in string_cols:
        existing[col] = existing[col].astype(str).str.strip()
    existing['quantity'] = existing['quantity'].astype(float)


    # 🔍 Deduplicate based on key fields
    key_cols = ['entry_date', 'invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number', 'quantity']
    merged = df.merge(existing, how='left', indicator=True, on=key_cols)
    print(f"🧾 merged:\n{merged.tail(25)}")
    new_rows = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

    print(f"🟡 Dry Run: {len(new_rows)} new rows would be inserted (out of {len(df)} total).")

    if not dry_run:
        new_rows.to_sql('receiving_log', engine, if_exists='append', index=False, method='multi')
        print("✅ Data inserted.")
    else:
        print("🚫 Dry run mode — no data inserted.")
        print("🔍 Preview of rows to be inserted:")
        print(new_rows.head(10))

def extract_wo_from_filename(filename):
    # Match either WO-20250970 or WO07-20250970
    match = re.search(r"WO\d{2}-(\d{8})", filename)
    if match:
        return match.group(1)
    return None

def validate_part_number_and_qty(file_path, df_sales_order):
    """
    Validates part numbers and quantities from a Word file against a filtered sales order.
    """
    word_parts = extract_product_details_from_word(file_path)
    filename = file_path.split("\\")[-1]
    wo_number = f'SO-{extract_wo_from_filename(filename)}'

    if not wo_number:
        print(f"❌ Could not extract WO number from filename: {filename}")
        return []

    df_filtered = df_sales_order[df_sales_order["WO_Number"] == wo_number]
    if df_filtered.empty:
        print(f"❌ No matching WO_Number found in sales order: {wo_number}")
        return []

    results = []
    for item in word_parts:
        word_part = item["product_number"]
        qty_text = item["qty"]

        try:
            expected_qty = int(qty_text)
        except ValueError:
            expected_qty = None

        matched_rows = df_filtered[
            df_filtered["Component"].apply(normalize) == normalize(word_part)
        ]

        if matched_rows.empty:
            results.append({
                "word_part": word_part,
                "status": "❓ NOT FOUND",
                "required_qty": None,
                "word_qty": expected_qty,
                "qty_match": "❌ N/A",
                "WO_Number": wo_number
            })
        else:
            db_qty = matched_rows["Required_Qty"].iloc[0]
            qty_match = expected_qty == db_qty if expected_qty is not None else False
            results.append({
                "word_part": word_part,
                "status": "✅ MATCH" if qty_match else "❌ QTY MISMATCH",
                "required_qty": db_qty,
                "word_qty": expected_qty,
                "qty_match": f"✅ Qty OK" if qty_match else f"❌ Expected {db_qty}, Got {expected_qty}",
                "WO_Number": wo_number
            })

    return results


## Parse PDF then upload to google sheet

def has_dash_comments(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if not hasattr(page, "annots") or page.annots is None:
                continue
            for annot in page.annots:
                content = annot.get("contents") or ""
                if "-" in content:
                    return True
    return False

def extract_pdf_data_with_dash_check(pdf_path):
    filename = os.path.basename(pdf_path).replace('.pdf', '')

    # Extract WO
    wo_match = re.search(r"WO-(\d{8}(?:r\d*)?)", filename)
    wo = wo_match.group(1) if wo_match else None

    # Extract PO
    po_match = re.search(r"(PO[#_])(\w+)", filename)
    po = po_match.group(2) if po_match else None

    # Extract customer
    customer = None
    if wo_match and po_match:
        try:
            customer_raw = filename.split(f"WO-{wo}_")[1].split(po_match.group(0))[0]
            customer = customer_raw.strip('_ ').replace('_', ' ')
        except Exception:
            customer = None

    dash_flag = has_dash_comments(pdf_path)

    if not all([wo, customer, po]):
        return wo, customer, po, pd.DataFrame(), dash_flag

    # Extract tables
    tables = []
    cleaned_header = ['Item', 'Description', 'Site', 'Ordered', 'To Pick']

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if not table:
                continue
            header = table[0]
            cleaned_raw_header = [str(col).strip() if col else "" for col in header]

            if cleaned_raw_header == cleaned_header:
                df = pd.DataFrame(table[1:], columns=cleaned_header)
                tables.append(df)
            elif 'P.O. No.' in cleaned_raw_header:
                try:
                    po_index = cleaned_raw_header.index('P.O. No.')
                    data_rows = [row[:po_index] + row[po_index+1:] for row in table[1:]]
                    cleaned_alt_header = cleaned_raw_header[:po_index] + cleaned_raw_header[po_index+1:]
                    if len(cleaned_header) == len(cleaned_alt_header):
                        df = pd.DataFrame(data_rows, columns=cleaned_header)
                        tables.append(df)
                except:
                    continue

    if not tables:
        return wo, customer, po, pd.DataFrame(), dash_flag

    # Clean
    final_df = pd.concat(tables, ignore_index=True)
    final_df.columns = final_df.columns.str.strip()
    final_df = final_df[final_df['Site'].astype(str).str.startswith('WH')].copy()

    def extract_or_default(row):
        col1 = row['Item']
        col2 = row['Description']

        def clean(text):
            return (
                text.strip()
                    .lstrip('*')
                    .replace("(cid:95)", "_")
                    .replace("(cid:74)", "J")
            )

        if isinstance(col1, str) and '...' in col1:
            if isinstance(col2, str):
                part = col2.strip().split('\n')[0].strip()
                return clean(part)
            return None
        elif isinstance(col1, str):
            return clean(col1)
        else:
            return None

    final_df['Extracted'] = final_df.apply(extract_or_default, axis=1)
    return wo, customer, po, final_df[['Extracted', 'Ordered']], dash_flag

def process_all_pdfs(folder_path):
    all_rows = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root, file)
                wo, customer, po, df, dash_flag = extract_pdf_data_with_dash_check(pdf_path)
                if df.empty:
                    continue
                for _, row in df.iterrows():
                    all_rows.append({
                        "Customer": customer,
                        "PO": po,
                        "WO": f"SO-{wo}",
                        "Product Number": row['Extracted'],
                        "Qty": row['Ordered'],
                        "Source File": file,
                        "Flagged": "⚠️" if dash_flag else ""
                    })

    return pd.DataFrame(all_rows)

def write_df_to_gsheet(df, sheet_name, worksheet_name, cred_path="path/to/your/credentials.json"):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(cred_path, scope)
    client = gspread.authorize(creds)

    sheet = client.open(sheet_name)
    try:
        worksheet = sheet.worksheet(worksheet_name)
        worksheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=worksheet_name, rows=1000, cols=20)

    set_with_dataframe(worksheet, df)
    print(f"✅ Data written to Google Sheet: {sheet_name} [{worksheet_name}]")

def components_for_wo(df, wo):
    # wo_norm = re.sub(r'^SO-', '', str(wo)).strip()
    wo_norm = wo
    return df[df['WO'] == wo_norm].sort_values('Product Number').reset_index(drop=True)




## Parse PDF. Merge PDF with Open Sales Order, keep Open Sales Order, Order by PDF
def reorder_df_out_by_output(output_df: pd.DataFrame, df_out: pd.DataFrame) -> pd.DataFrame:
    # 1) Build the reference order from output_df (the order you trust)
    ref = output_df.copy()
    ref['__pos_out'] = ref.groupby('WO').cumcount()                 # position within WO
    ref['__occ'] = ref.groupby(['WO','Product Number']).cumcount()  # occurrence index for duplicates
    ref_key = ref[['WO','Product Number','__occ','__pos_out']]

    # 2) On df_out, tag each duplicate with its own occurrence index
    tgt = df_out.copy()
    tgt['__occ'] = tgt.groupby(['WO','Product Number']).cumcount()

    # 3) Merge positions from output_df to df_out rows (match by WO + Product + occurrence)
    merged = tgt.merge(ref_key, on=['WO','Product Number','__occ'], how='left')

    # 4) For rows not present in output_df, keep their original within-WO order but push them after the matched ones
    merged['__fallback'] = merged.groupby('WO').cumcount()
    merged['__pos_out'] = merged['__pos_out'].fillna(np.inf)

    # 5) Final order: by WO, then by output_df position; if missing, by original order
    ordered = (merged
               .sort_values(['WO','__pos_out','__fallback'])
               .drop(columns=['__occ','__pos_out','__fallback'])
               .reset_index(drop=True))
    return ordered


# Parse PDF

In [ ]:
# Run
folder = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Work Order- WO"
google_cred_path = r"C:\Users\Admin\Downloads\pdfwo-466115-734096e1cef8.json"

output_df = process_all_pdfs(folder)

if not output_df.empty:
    write_df_to_gsheet(
        output_df,
        sheet_name="PDF_WO",
        worksheet_name="Sheet1",
        cred_path=google_cred_path
    )
else:
    print("❌ No output to write.")

## Parse Open Sales Order.CSV

In [94]:
df_sales_order.rename(columns={
    'Unnamed: 0': 'Component',
    'Num': 'WO_Number',
    'Qty': 'Required_Qty'
}, inplace=True)

df_sales_order['Component'] = df_sales_order['Component'].ffill().astype(str).str.strip()
df_sales_order = df_sales_order[~df_sales_order["Component"].str.startswith("total")]
# df_sales_order["WO_Number"] = (
#     df_sales_order["WO_Number"].astype(str).str.strip().str.replace(r'^SO-', '', regex=True)
# )
df_sales_order = df_sales_order[
    ~df_sales_order["Component"].str.lower().isin(["forwarding charge", "tariff (estimation)"])
]

# ---------- Format like your screenshot ----------
# If your CSV already has Customer / PO / Source File columns, these will pass through.
# If not, we create empty placeholders so the table layout matches.
needed_cols = {
    'Name': 'Customer',
    'P. O. #': 'PO',
    'WO_Number': 'WO',
    'Component': 'Product Number',
    'Required_Qty': 'Qty',
    'Source File': 'Source File'
}
for c in ['Customer', 'PO', 'Source File']:
    if c not in df_sales_order.columns:
        df_sales_order[c] = ""

df_out = df_sales_order.rename(columns=needed_cols)[list(needed_cols.values())]

# Sort to group visually by WO, then by Product Number
df_out = df_out.sort_values(['WO', 'Product Number']).reset_index(drop=True)

# Usage:
final_df = reorder_df_out_by_output(output_df, df_out)


# your specific mappings (only exact string-to-string)
mappings = {
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-512GB-PCIe34-TLC5WT-T': 'M.242-SSD-512GB-PCIe34-TLC5WT-TD',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
}

final_df['Product Number'] = final_df['Product Number'].replace(mappings)

final_df = final_df.loc[:, ~final_df.columns.duplicated()]

# Upload to Google Sheet
google_cred_path = r"C:\Users\Admin\Downloads\pdfwo-466115-734096e1cef8.json"

if not final_df.empty:
    write_df_to_gsheet(
        final_df,
        sheet_name="PDF_WO",
        worksheet_name="Open Sales Order",
        cred_path=google_cred_path
    )
else:
    print("❌ No output to write.")


# ✅ Now final_df has Product Number overridden from df_out when WO matches and value differs
final_df[final_df['WO'] == 'SO-20250494']

✅ Data written to Google Sheet: PDF_WO [Open Sales Order]


,Customer,PO,WO,Product Number,Qty,Source File
151,Aerflite Canada Inc.,564,SO-20250494,CS-RM44-601,10.0,
152,Aerflite Canada Inc.,564,SO-20250494,LC-XE360-SP5-601,10.0,
153,Aerflite Canada Inc.,564,SO-20250494,PA-HA1300R-PM-601,10.0,
154,Aerflite Canada Inc.,564,SO-20250494,RDDR5-32GB-ECC48WT-TD-601,40.0,
155,Aerflite Canada Inc.,564,SO-20250494,GC-RTX6000Ada-PNY-601,10.0,
156,Aerflite Canada Inc.,564,SO-20250494,Engineer Services,10.0,
157,Aerflite Canada Inc.,564,SO-20250494,MB-GENOAD8X-2T/BCM-9534-SSD-601,10.0,


# Load Receiving Log into DB

In [9]:
if __name__ == '__main__':
    # 1️⃣ Push the Flask context so db.engine is wired up:
    with app.app_context():
        # 2️⃣ Ensure the receiving_log table exists
        db.create_all()

        # 3️⃣ Now call your loader, handing it db.engine
        load_receiving_log(
            r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Receiving Log_ZC_2.0.xlsm",
            db.engine
        )

🧾 Cleaned DataFrame Preview:
     entry_date   invoice_number box_number  pod_number part_number  \
2042 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2043 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2044 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2045 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2046 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2047 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2048 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2049 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2050 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2051 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2052 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2053 2025-08-11  Newegg Business       1--1  POD-251120    i7-14700   
2054 2025-08-11  Newegg Business       1--1  POD

# Go Through Today's WOs

In [98]:
import os
import logging
from datetime import datetime, timedelta

def process_all_wo_files(folder_path, df_sales_order):
    today = datetime.today().date() - timedelta(days=0)
    all_results = {}

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".docx"):
                file_path = os.path.join(root, file)
                creation_time = datetime.fromtimestamp(os.path.getctime(file_path)).date()

                if creation_time == today:
                    logging.info(f"\n📄 Processing file: {file}")

                    # --- SN Validation ---
                    sn_results = validate_sn_part_matches_via_api(file_path)

                    # --- Part Number & Qty Validation ---
                    qty_results = validate_part_number_and_qty(file_path, df_sales_order)

                    combined_results = {
                        "serial_validation": sn_results,
                        "part_qty_validation": qty_results
                    }

                    all_results[file] = combined_results

                    # --- Logging Output ---
                    if sn_results:
                        logging.info("🔍 Serial Number Validation:")
                        for r in sn_results:
                            logging.info(f"SN: {r['serial_number']} | Word Part: {r['word_part']} | "
                                         f"DB Part: {r['db_part']} | Status: {r['status']} | "
                                         f"Qty Check: {r['qty_check']}")
                    else:
                        logging.warning("⚠️ No SN results.")

                    if qty_results:
                        logging.info("📦 Part Number + Qty Validation:")
                        for r in qty_results:
                            logging.info(f"Part: {r['word_part']} | WO: {r.get('WO_Number')} | "
                                         f"Status: {r['status']} | Qty Check: {r['qty_match']}")
                    else:
                        logging.warning("⚠️ No Part/QTY results.")

                    logging.info("\n" + "-"*60 + "\n")

    return all_results

def normalize(s):
    """Standardize part number or component string for reliable comparison."""
    return str(s).strip().upper().replace("-", "").replace(" ", "")



# Run validations
folder = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\02 Work Order- Word file\Work Order 2025\Work Order 2025-08"
results = process_all_wo_files(folder, df_sales_order)


INFO:root:
📄 Processing file: WO08-20251009-American Portwell Technology, Inc..docx
INFO:root:🔍 Serial Number Validation:
INFO:root:SN: P2400022 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2400023 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2600525 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2600526 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2800351 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2800352 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2800353 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P2800354 | Word Part: PCIe-N572 | DB Part: PCIe-N572 | Status: ✅ MATCH | Qty Check: ✅ Qty OK
INFO:root:SN: P3000460

## Debug
-
-
-
-
-









## Go Through WO

In [100]:
file_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\02 Work Order- Word file\Work Order 2025\Work Order 2025-08\WO08-20250893r-RDI Technologies.docx"

# First, extract product details from the Word document
product_details = extract_product_details_from_word(file_path)

# Now pass the file_path to the validation function
results = validate_sn_part_matches_via_api(file_path)

# Print the results
for r in results:
    print(f"SN: {r['serial_number']} | Word: {r['word_part']} | DB: {r['db_part']} | {r['status']} |  {r['qty_check']}")

SN: P2900820 | Word: Nuvo-9006DE | DB: Nuvo-9006DE | ✅ MATCH |  ✅ Qty OK
SN: P2900821 | Word: Nuvo-9006DE | DB: Nuvo-9006DE | ✅ MATCH |  ✅ Qty OK
SN: P2900824 | Word: Nuvo-9006DE | DB: Nuvo-9006DE | ✅ MATCH |  ✅ Qty OK
SN: P2900825 | Word: Nuvo-9006DE | DB: Nuvo-9006DE | ✅ MATCH |  ✅ Qty OK
SN: P2900826 | Word: Nuvo-9006DE | DB: Nuvo-9006DE | ✅ MATCH |  ✅ Qty OK
SN: P2900827 | Word: Nuvo-9006DE | DB: Nuvo-9006DE | ✅ MATCH |  ✅ Qty OK
SN: U3XL329400497, X336H792 | Word: i9-14900 | DB: i9-14900 | ✅ MATCH |  ✅ Qty OK
SN: U3EP689002733, X344P526 | Word: i9-14900 | DB: i9-14900 | ✅ MATCH |  ✅ Qty OK
SN: M4137QR803356, L437F823 | Word: i9-14900 | DB: i9-14900 | ✅ MATCH |  ✅ Qty OK
SN: U4GB082300134, X427M472 | Word: i9-14900 | DB: i9-14900 | ✅ MATCH |  ✅ Qty OK
SN: U4XN682803199, X412N124 | Word: i9-14900 | DB: i9-14900 | ✅ MATCH |  ✅ Qty OK
SN: U4WH748303353, X412N186 | Word: i9-14900 | DB: i9-14900 | ✅ MATCH |  ✅ Qty OK
SN: NCA12407090430054 | Word: DDR5-16GB-WT48-IK | DB: None | ❓ NOT FOU

## Update changes in excel to Supabase (IN# = IN250225004)

In [ ]:
# def replace_invoice_rows(file_path, invoice_no):
#     import pandas as pd

#     # Load and clean the Excel
#     df = pd.read_excel(file_path, sheet_name="Receiving")
#     df = df.rename(columns={
#         'Date': 'entry_date',
#         'Inv# ': 'invoice_number',
#         'Box #': 'box_number',
#         'POD#': 'pod_number',
#         'Part#': 'part_number',
#         'SN#': 'serial_number',
#         'QTY': 'quantity'
#     })

#     df['quantity'] = df['quantity'].fillna(1).astype(float)
#     df['entry_date'] = pd.to_datetime(df['entry_date'], errors='coerce')

#     string_cols = ['invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number']
#     for col in string_cols:
#         df[col] = df[col].astype(str).str.strip()

#     # Filter only rows for the target invoice
#     df_target = df[df['invoice_number'] == invoice_no].copy()

#     with app.app_context():
#         with db.engine.begin() as conn:
#             # Delete existing rows for that invoice
#             conn.execute(
#                 db.text("DELETE FROM receiving_log WHERE invoice_number = :inv"),
#                 {"inv": invoice_no}
#             )
#             # Insert the cleaned rows from Excel
#             df_target.to_sql('receiving_log', conn, if_exists='append', index=False)

#     print(f"✅ Replaced invoice {invoice_no} with {len(df_target)} rows from Excel.")


# if __name__ == '__main__':
#     replace_invoice_rows(
#         r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Receiving Log_ZC.xlsm",
#         "IN250225004"
#     )



✅ Replaced invoice IN250225004 with 302 rows from Excel.


## Parse PDF 
#### (Before 20250187: [None, None, 'P.O. No.', 'Ship Date', None, 'Expected L/T', None] )
#### (After 20250187: ['Item', 'Description', 'Site', 'Ordered', 'To Pick'])

In [ ]:
import pdfplumber
import pandas as pd


Path = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Work Order- WO\WO-20250957r_CACI Federal_PO_P000214574_(2-2).pdf"

filename = os.path.basename(Path)

Cleaned_header = ['Item', 'Description', 'Site', 'Ordered', 'To Pick']

with pdfplumber.open(Path) as pdf:
    match = re.search(r"WO-(\d{6,8}[a-zA-Z]?)_([A-Za-z0-9\s&\-]+)_PO_(P\d+)", filename)
    if match:
        wo = match.group(1)             # '20250957r'
        customer = match.group(2)       # 'CACI Federal'
        po = match.group(3)             # 'P000214574'

        print("WO:", wo)
        print("Customer:", customer)
        print("PO:", po)
    else:
        print("Pattern not matched.")


    tables = []
    for page in pdf.pages:
        table = page.extract_table()
        if not table:
            continue

        header = table[0]
        cleaned_raw_header = [str(col).strip() if col else "" for col in header]

        if cleaned_raw_header == Cleaned_header:
            # Standard case
            df = pd.DataFrame(table[1:], columns=Cleaned_header)
            tables.append(df)

        elif 'P.O. No.' in cleaned_raw_header:
            try:
                po_index = cleaned_raw_header.index('P.O. No.')
                # Remove the column from all rows
                data_rows = [row[:po_index] + row[po_index+1:] for row in table[1:]]
                # Remove from header too
                cleaned_alt_header = cleaned_raw_header[:po_index] + cleaned_raw_header[po_index+1:]

                # Force assign Cleaned_header if lengths match
                if len(Cleaned_header) == len(cleaned_alt_header):
                    df = pd.DataFrame(data_rows, columns=Cleaned_header)
                    tables.append(df)
                else:
                    print("Header length mismatch after removing 'P.O. No.':", cleaned_alt_header)
            except Exception as e:
                print("Error processing alternate header table:", e)

        else:
            print("Unknown table format:", cleaned_raw_header)


# Combine all extracted tables
final_df = pd.concat(tables, ignore_index=True)

# Strip column names of trailing spaces
final_df.columns = final_df.columns.str.strip()

# Replace any odd encoding
for col in final_df.select_dtypes(include='object').columns:
    final_df[col] = final_df[col].map(lambda x: x.replace("(cid:95)", "_") if isinstance(x, str) else x)

# Filter rows where 'Site' starts with 'WH'
filtered_df = final_df[final_df['Site'].astype(str).str.startswith('WH')].copy()

def extract_or_default(row):
    col1 = row['Item']
    col2 = row['Description']
    if isinstance(col1, str) and '...' in col1:
        if isinstance(col2, str):
            match = re.search(r'\*(.*?)\n', col2)
            if match:
                return match.group(1)
        return None
    else:
        return col1

filtered_df['Extracted'] = filtered_df.apply(extract_or_default, axis=1)
print(filtered_df[['Extracted','Ordered']])


WO: 20250957r
Customer: CACI Federal
PO: P000214574
                         Extracted Ordered
4                       SEMIL-2007       6
5                         i9-14900       6
8                  DDR5-32GB-48-SM      12
9   M.280-SSD-2TB-PCIe44-TLC5WT-TD       6
11            Cblkit-M12-SEMIL2000       6
15          Cbl-TpCPlug-UTpCF-50CM       6
16           Cbl-TpCPlug-U3TA-50CM       6
17              Cbl-TpCPlug-DPM-1M       6
19                 PA-280W-CW6P-2P       6
20                Cbl-PC-TW-180CM1       6


In [ ]:
def extract_pdf_data(pdf_path):
    filename = os.path.basename(pdf_path)
    name_without_ext = filename.replace('.pdf', '')
    parts = name_without_ext.split('_')
    wo_match = re.match(r"WO-(\d{8}[a-zA-Z]?)", parts[0])
    if wo_match and len(parts) >= 3 and parts[-2] == "PO":
        wo = wo_match.group(1)
        customer = "_".join(parts[1:-2]).replace('_', ' ')  # Handles multi-word customer
        po = parts[-1]
    

    tables = []
    cleaned_header = ['Item', 'Description', 'Site', 'Ordered', 'To Pick']
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if not table:
                continue
            header = table[0]
            cleaned_raw_header = [str(col).strip() if col else "" for col in header]

            if cleaned_raw_header == cleaned_header:
                df = pd.DataFrame(table[1:], columns=cleaned_header)
                tables.append(df)
            elif 'P.O. No.' in cleaned_raw_header:
                try:
                    po_index = cleaned_raw_header.index('P.O. No.')
                    data_rows = [row[:po_index] + row[po_index+1:] for row in table[1:]]
                    cleaned_alt_header = cleaned_raw_header[:po_index] + cleaned_raw_header[po_index+1:]
                    if len(cleaned_header) == len(cleaned_alt_header):
                        df = pd.DataFrame(data_rows, columns=cleaned_header)
                        tables.append(df)
                except:
                    continue

    if not tables:
        return wo, customer, po, pd.DataFrame()
    
    final_df = pd.concat(tables, ignore_index=True)
    final_df.columns = final_df.columns.str.strip()

    # Filter after concat
    final_df = final_df[final_df['Site'].astype(str).str.startswith('WH')].copy()

    def extract_or_default(row):
        col1 = row['Item']
        col2 = row['Description']
        if isinstance(col1, str) and '...' in col1:
            if isinstance(col2, str):
                match = re.search(r'\*(.*?)\n', col2)
                if match:
                    return match.group(1)
            return None
        else:
            return col1

    final_df['Extracted'] = final_df.apply(extract_or_default, axis=1)
    return wo, customer, po, final_df[['Extracted', 'Ordered']]




df = extract_pdf_data(r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Work Order- WO\WO-20250923_CoastIPC_PO_P96448.pdf")
df


('20250923',
 'CoastIPC',
 'P96448',
             Extracted Ordered
 3  Nuvo-9166GC-PoE-UL       1
 4           i9-13900E       1)

In [22]:
output_df

,Customer,PO,WO,Product Number,Qty,Source File,Flagged
0,GPR,1026,SO-20250001,Nuvo-7204VTC,5,WO-20250001_GPR_PO#1026.pdf,
1,GPR,1026,SO-20250001,i7-8700T,5,WO-20250001_GPR_PO#1026.pdf,
2,GPR,1026,SO-20250001,DDR4-32GB-WT32-SM,5,WO-20250001_GPR_PO#1026.pdf,
3,GPR,1026,SO-20250001,SSD-256GB-TLCWT-IK,5,WO-20250001_GPR_PO#1026.pdf,
4,GPR,1026,SO-20250001,SSD-2TB-TLC5WT-TD,5,WO-20250001_GPR_PO#1026.pdf,
...,...,...,...,...,...,...,...
6167,Intelligent Wellhead(Canada),9001,SO-20251149,i9-13900,6,WO-20251149_Intelligent Wellhead(Canada)_PO_90...,
6168,Intelligent Wellhead(Canada),9001,SO-20251149,GC-RTX4000SFFAda-PNY,6,WO-20251149_Intelligent Wellhead(Canada)_PO_90...,
6169,Intelligent Wellhead(Canada),9001,SO-20251149,DDR5-32GB-48WT-SM,12,WO-20251149_Intelligent Wellhead(Canada)_PO_90...,
6170,Intelligent Wellhead(Canada),9001,SO-20251149,SSD-4TB-TLC5WT-TD,6,WO-20251149_Intelligent Wellhead(Canada)_PO_90...,


In [74]:
import pandas as pd
import numpy as np

def reorder_df_out_by_output(output_df: pd.DataFrame, df_out: pd.DataFrame) -> pd.DataFrame:
    # 1) Build the reference order from output_df (the order you trust)
    ref = output_df.copy()
    ref['__pos_out'] = ref.groupby('WO').cumcount()                 # position within WO
    ref['__occ'] = ref.groupby(['WO','Product Number']).cumcount()  # occurrence index for duplicates
    ref_key = ref[['WO','Product Number','__occ','__pos_out']]

    # 2) On df_out, tag each duplicate with its own occurrence index
    tgt = df_out.copy()
    tgt['__occ'] = tgt.groupby(['WO','Product Number']).cumcount()

    # 3) Merge positions from output_df to df_out rows (match by WO + Product + occurrence)
    merged = tgt.merge(ref_key, on=['WO','Product Number','__occ'], how='left')

    # 4) For rows not present in output_df, keep their original within-WO order but push them after the matched ones
    merged['__fallback'] = merged.groupby('WO').cumcount()
    merged['__pos_out'] = merged['__pos_out'].fillna(np.inf)

    # 5) Final order: by WO, then by output_df position; if missing, by original order
    ordered = (merged
               .sort_values(['WO','__pos_out','__fallback'])
               .drop(columns=['__occ','__pos_out','__fallback'])
               .reset_index(drop=True))
    return ordered


# Usage:
final_df = reorder_df_out_by_output(output_df, df_out)


# your specific mappings (only exact string-to-string)
mappings = {
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-512GB-PCIe34-TLC5WT-T': 'M.242-SSD-512GB-PCIe34-TLC5WT-TD',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
}

final_df['Product Number'] = final_df['Product Number'].replace(mappings)


# ✅ Now final_df has Product Number overridden from df_out when WO matches and value differs
final_df[final_df['WO'] == 'SO-20250843']

,Customer,Customer,PO,PO,WO,Product Number,Qty,Source File
354,Boston Scientific,,7000580842,,SO-20250843,Nuvo-9006DE-PoE-UL,1.0,
355,Boston Scientific,,7000580842,,SO-20250843,i9-13900,1.0,
356,Boston Scientific,,7000580842,,SO-20250843,DDR5-16GB-48-SM,2.0,
357,Boston Scientific,,7000580842,,SO-20250843,M.280-SSD-256GB-PCIe44-TLC5WT-TD,1.0,
358,Boston Scientific,,7000580842,,SO-20250843,SSD-1TB-TLC5ET-TD,1.0,
359,Boston Scientific,,7000580842,,SO-20250843,Dust Cover-DP,1.0,
360,Boston Scientific,,7000580842,,SO-20250843,Dust Cover-COM,4.0,
361,Boston Scientific,,7000580842,,SO-20250843,Dust Cover-USB-AF,8.0,
362,Boston Scientific,,7000580842,,SO-20250843,Dust Cover-USB-CF,1.0,
363,Boston Scientific,,7000580842,,SO-20250843,Dust Cover-DVI,1.0,
